# What is the makeup of people who have adjusted the state of the Discussion tools beta feature?

[Task](https://phabricator.wikimedia.org/T297714)

To inform the design of the design of the New Discussion Tools Beta feature, we are interested in determining the following: Of the people who have explicitly adjusted the state of the Discussion tools beta feature within Special:Preferences#mw-prefsection-betafeatures, what percentage of these people are Junior Contributors and what percentage of these people are Senior Contributors?

In [1]:
import pandas as pd
import numpy as np

import datetime as dt

from wmfdata import hive, mariadb

In [4]:
HIVE_SNAPSHOT = "2021-11"
START_OF_DATA = "2021-02-18"
END_OF_DATA = "2021-12-01"

In [88]:
# Collect all users that have made an edit since deployment of the tool grouped by experience
query = """

SELECT
    event_user_id as contributor,
    wiki_db as wiki,
    CASE
        WHEN min(event_user_revision_count) < 100 THEN 'under 100'
        WHEN (min(event_user_revision_count) >=100 AND min(event_user_revision_count <= 500)) THEN '100-500'
        ELSE 'over 500'
        END AS edit_count_group
FROM wmf.mediawiki_history AS mh
WHERE 
    snapshot = '2021-11' 
-- date first deployed
    AND event_timestamp >= '2021-02-18'  
    AND event_timestamp < '2021-12-01'   
-- only on desktop
    AND NOT array_contains(revision_tags, 'iOS')
    AND NOT array_contains(revision_tags, 'Android')
    AND NOT array_contains(revision_tags, 'Mobile Web')
    AND event_entity = 'revision' AND 
    event_type = 'create'
    AND event_user_is_anonymous = FALSE
   -- find all edits on talk pages 
    AND page_namespace_historical % 2 = 1
    AND SIZE(event_user_is_bot_by_historical) = 0  -- not a bot
    AND NOT (revision_is_identity_reverted AND 
            revision_seconds_to_identity_revert <= 172800) -- not reverted within 48 hours
GROUP BY
    event_user_id,
    wiki_db
"""

In [89]:
contributors = hive.run(
    query.format(
        hive_snapshot = HIVE_SNAPSHOT,
        START_OF_DATA= START_OF_DATA,
        END_OF_DATA=END_OF_DATA
    )
)

In [90]:
#Total contributors

Total_contributors = contributors['contributor'].count()

print('Total number of contributors for whom we will be checking beta preferences:' , Total_contributors)

Total number of contributors for whom we will be checking beta preferences: 355945


In [91]:
contributors.head()

,contributor,wiki,edit_count_group
0,153,mediawikiwiki,100-500
1,191,suwiktionary,100-500
2,1084,lawiki,over 500
3,1173,plwiki,over 500
4,2009,uawikimedia,over 500


# Discussion Tool Beta Preference

Find the status of the discussion tool beta preference for all contributors

## Quick look at all users (not just contributors) that have adjusted the feature

In [81]:
query = """ 
    SELECT
        DISTINCT database_code
    FROM canonical_data.wikis
    WHERE
    database_code NOT IN ('labswiki', 'labtestwiki')
   """

In [82]:
wikis = hive.run(query)

In [83]:
wiki_list=wikis['database_code'].unique()

In [84]:
query='''
SELECT COUNT(*) 
FROM user_properties 
WHERE up_property="discussiontools-betaenable"
'''

In [85]:
all_dt_beta_users = mariadb.run(commands = query, dbs = wiki_list, use_x1=False, format="pandas")

In [87]:
all_dt_beta_users.sum()

COUNT(*)    290842
dtype: int64

There are a total of 290,842 users that have explicitly adjusted the beta feature across all wikis.  

## Discussion Tool Beta Feature Preference by Talk Page Contributors

In [92]:
query='''
SELECT 
  user_properties.up_value AS preference,
  user_properties.up_user AS `user`
FROM user_properties 
WHERE user_properties.up_user in ({users})
AND up_property = "discussiontools-betaenable"
'''    

In [93]:
# Looping through each wiki to find users that have the discussiontools-beta enable preference set

wikis = contributors['wiki'].unique()
up_pref=list()
for wiki in wikis:
    user_ids = contributors[contributors['wiki'] == wiki]["contributor"]
    user_list = ','.join([str(u) for u in user_ids])
    prefs = mariadb.run(
      query.format(users=user_list),
      wiki
    )
    up_pref.append(prefs)

pref= pd.concat(up_pref)

In [94]:
# Join with edit count data from mediawiki_history
contributor_beta_pref = contributors.join(pref.set_index('user'), on = 'contributor', how = 'left')

In [95]:
contributor_beta_pref.head()

,contributor,wiki,edit_count_group,preference
0,153,mediawikiwiki,100-500,b'0'
0,153,mediawikiwiki,100-500,b'0'
1,191,suwiktionary,100-500,b'0'
2,1084,lawiki,over 500,NaN
3,1173,plwiki,over 500,b'1'


In [96]:
# convert preference column to string type 
contributor_beta_pref['preference'] = contributor_beta_pref['preference'].astype(str)

In [97]:
# rename values. 
pref_aliases = {
    "b\'0\'":"explicitly disabled beta preference",
    "b\'1\'":"explicitly enabled beta preference",
    "nan": "no local preference recorded"
}

contributor_beta_pref= contributor_beta_pref.replace({"preference": pref_aliases})

In [98]:
# Overall Opt Out Rate

In [99]:
contributor_beta_pref_overall= contributor_beta_pref[['preference', 'contributor']].groupby('preference').count()

contributor_beta_pref_overall

,contributor
preference,
explicitly disabled beta preference,44546
explicitly enabled beta preference,18690
no local preference recorded,295215


In [100]:
pct_beta_opt_rate =(100. * contributor_beta_pref_overall / contributor_beta_pref_overall.sum()).round(2).astype(str) + '%'
pct_beta_opt_rate.sort_values(by=['contributor'],ascending=False)

,contributor
preference,
no local preference recorded,82.36%
explicitly enabled beta preference,5.21%
explicitly disabled beta preference,12.43%


Of all talk page contributors, around 17% have explicitly adjusted the DiscussionTools Beta Preference since deployed.

# By Experience Level

We are only interested in the makeup of users that have explcitily adjusted the DT beta preference. 

In [101]:
contributor_beta_pref.head()

,contributor,wiki,edit_count_group,preference
0,153,mediawikiwiki,100-500,explicitly disabled beta preference
0,153,mediawikiwiki,100-500,explicitly disabled beta preference
1,191,suwiktionary,100-500,explicitly disabled beta preference
2,1084,lawiki,over 500,no local preference recorded
3,1173,plwiki,over 500,explicitly enabled beta preference


In [102]:
contributor_beta_pref_adjusted = contributor_beta_pref[contributor_beta_pref['preference'] != 'no local preference recorded']

# Percentages of all talk page contributors that have adjusted the beta preference

In [104]:
# calculate total enabled and disabled
contributor_beta_pref_byexp = contributor_beta_pref_adjusted[['edit_count_group' ,'contributor']].groupby(['edit_count_group']).count()

contributor_beta_pref_byexp

,contributor
edit_count_group,
100-500,6800
over 500,23812
under 100,32624


In [105]:
pct_beta_opt_rate_byexp =(100. * contributor_beta_pref_byexp/ contributor_beta_pref_byexp.sum()).round(2).astype(str) + '%'
pct_beta_opt_rate_byexp

,contributor
edit_count_group,
100-500,10.75%
over 500,37.66%
under 100,51.59%
